In [2]:
import pickle
import re
import numpy as np

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

age = []
fc = []
sex = []
race = []
res1 = []
res5 = []
ab1 = []
ab5 = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

for sub in demo['age_at_cnb']:
    for task in ['rest', 'nback', 'emoid']:
        if sub not in demo['Race'] or demo['Race'][sub] not in ['AA', 'EA']:
            continue
        a = demo['age_at_cnb'][sub]
        s = demo['Sex'][sub] == 'M'
        r = demo['Race'][sub] == 'AA'
        try:
            p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
        except:
            continue
        r1 = np.load(f'{demodir}/fc/{sub}_task-{task}res1_fc.npy')
        res1.append(r1)
        r5 = np.load(f'{demodir}/fc/{sub}_task-{task}res5_fc.npy')
        res5.append(r5)
        a1 = np.load(f'{demodir}/fc/{sub}_task-{task}ab1_fc.npy')
        ab1.append(a1)
        a5 = np.load(f'{demodir}/fc/{sub}_task-{task}ab5_fc.npy')
        ab5.append(a5)
        age.append(a)
        sex.append(s)
        race.append(r)
        fc.append(p)
    
race = np.array(race)
sex = np.array(sex)
age = np.array(age)
fc = np.stack(fc)
res1 = np.stack(res1)
res5 = np.stack(res5)
ab1 = np.stack(ab1)
ab5 = np.stack(ab5)

print([a.shape for a in [sex, race, age, fc, res1, res5, ab1, ab5]])
print(np.mean(age))
print(np.mean(sex))
print(np.mean(race))

[(3849,), (3849,), (3849,), (3849, 34716), (3849, 34716), (3849, 34716), (3849, 34716), (3849, 34716)]
14.398285268901013
0.473629514159522
0.4811639386853728


In [3]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

# aa = np.where(race == 0)[0]
# m = np.where(sex == 0)[0]
# aam = np.intersect1d(aa, m)
# idcs = np.random.permutation(aam.shape[0])[:807]
# aam = aam[idcs]
aam = np.random.permutation(fc.shape[0])[:807]
print(aam.shape)

accs = []

for i in range(20):
    x = res1[aam]
    xx = fc[aam]
    y = age[aam]
    
    xtr, xt, xxtr, xxt, ytr, yt = train_test_split(x, xx, y, train_size=0.8)
    
    mu = np.mean(ytr)
    ytr = ytr-mu
    yt = yt-mu
    
    reg = Ridge(alpha=1).fit(xtr, ytr)
    yhat0 = reg.predict(xt)
    reg = Ridge(alpha=1).fit(xxtr, ytr)
    yhat1 = reg.predict(xxt)
    acc = rmse((yhat0+yhat1)/2, yt)
    accs.append(acc)
    print(acc)
    
print('---')
print(np.mean(accs))
print(np.std(accs))


(807,)
2.3103851352712947
2.1972192012798484
2.299388065134683
2.075893472374327
2.1873835898469856
2.259958319196827
2.3859832102301386
2.1111835826247987
2.238894730640563
1.939601139025712
2.322702143635315
2.0422361739333574
2.3405759206909664
2.1885041267202676
2.1957926925155506
2.24076825189534
2.139236786306959
2.159162397645103
2.120457449912572
2.2559227543400415
---
2.2005624571610327
0.10708695988216937


In [38]:
print(np.std(accs))

0.10154195352626498


In [65]:
print(rmse(0, yt))

3.486050913704901
